In [2]:
import os
import re
import torch
from difflib import SequenceMatcher
from langchain.chains import LLMChain
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

d:\hf\code\data_code\nl2sql\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
base_model = LlamaForCausalLM.from_pretrained(
     "mistralai/Mistral-7B-Instruct-v0.1",
     load_in_8bit=True,
     device_map='auto',
    )
tokenizer = LlamaTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
pipe = pipeline(
        "text-generation",
        model=base_model,
        tokenizer=tokenizer,
        max_length=500,
        temperature=0.3,
        top_p=0.95,
        repetition_penalty=1.2
    )
local_llm = HuggingFacePipeline(pipeline=pipe)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1.
401 Client Error. (Request ID: Root=1-67542227-594b1c1a649b56c12ff38312;cc68c2cf-c42c-4bad-9cc2-808a2b74c1b5)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
def find_columns_match(question, input_dict):
try:
  question_list = re.split(r'\s|,|\.', question)
  for index, string2 in enumerate(question_list):
    for string1 in input_dict.get('table1_columns'):
      score = SequenceMatcher(None,string1.lower(), string2.lower()).ratio()*100
      if score > 91:
        question_list[index] = string1 + ","
  return " ".join(question_list)
  
except:
 return question


In [ ]:
def query_generator(tble,cols,question):

  template = """Generate a SQL query using the following table name: {Table}, and columns as a list: {Columns}, to answer the following question:
  {question}.
  
  Output Query:
  
  """
  
  prompt = PromptTemplate(template=template, input_variables=["Table","question","Columns"])
  
  llm_chain = LLMChain(prompt=prompt, llm=local_llm)
  
  response= llm_chain.run({"Table" : tble,"question" :question, "Columns" : cols})
  print(response)


In [ ]:
transaction = [
        "transaction_id",
        "transaction_amount",
        "transaction_date",
        "transaction_type",
        "transaction_status",
        "transaction_description",
        "transaction_source_account",
        "transaction_destination_account",
        "transaction_currency",
        "transaction_fee"
    ]

inputs = ["Generate an SQL query to retrieve transaction_amount, transaction_date, transaction_type,transaction_description where transaction_id is 10",
         "Generate an SQL query to retrieve transaction_id, transaction_date, transaction_type,transaction_source_account where transaction_status is 'completed'",
         "Generate an SQL query to retrieve count of the transaction_type and their average transaction_amount, ordered by transaction_type.",
         "Generate an SQL query to retrieve list of the total transaction amount for each source account, sorted by total transaction amount in descending order.",
         "Generate an SQL query to retrieve find the maximum transaction amount for each transaction type, ordered by transaction type."]

for input in inputs:
    query_generator("transaction",transaction ,question=find_columns_match(input,transaction))
